In [2]:
import numpy as np
from PIL import Image
import os
import cv2 as cv
from skimage.feature import hog
from skimage.transform import resize
from skimage.io import imread, imshow, imsave

np.random.seed(42)

In [4]:
import zipfile
import glob

file = glob.glob('./clothing-dataset-small-master.zip')

with zipfile.ZipFile(file[0], 'r') as zip_ref:
    zip_ref.extractall('data/')

In [ ]:
for data_type in ["train", "validation", "test"]:
    folder_path = "./data/clothing-dataset-small-master/" + data_type
    
    # combine validation and train
    new_folder_path = "./data/clothing-dataset/" + ("test" if data_type == "test" else "train")

    for folder in os.listdir(folder_path):
        new_path = os.path.join(new_folder_path, folder)
        if not os.path.exists(new_path):
            os.makedirs(new_path)

        for filename in os.listdir(os.path.join(folder_path, folder)):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder_path, folder, filename)
                img = Image.open(image_path)
                new_image_path = os.path.join(new_folder_path, folder, filename)
                img.save(new_image_path)
                                              
                # flip all images besides t-shirts to balance the data
                if folder != "t-shirt":
                    img = img.transpose(Image.FLIP_LEFT_RIGHT)
                    new_image_path = os.path.join(new_folder_path, folder, "flipped_" + filename)
                    img.save(new_image_path)

In [ ]:
# Generate HOG data from above data
data_types = ["train", "test"]

for data_type in data_types:
    folder_path = "./data/clothing-dataset/" + data_type
    new_folder_path = "./data/clothing-dataset-hog/" + data_type

    for folder in os.listdir(folder_path):
        new_path = os.path.join(new_folder_path, folder)
        if not os.path.exists(new_path):
            os.makedirs(new_path)

        for filename in os.listdir(os.path.join(folder_path, folder)):
            # Open the image
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder_path, folder, filename)
                img = cv.imread(image_path,cv.IMREAD_GRAYSCALE)
                resized_img  = cv.resize(img, (64, 128),interpolation =cv.INTER_LINEAR)
                fd, hog_image = hog(resized_img , orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True)
                new_image_path = os.path.join(new_folder_path, folder, filename)
                cv.imwrite(new_image_path, hog_image)

In [ ]:
# Generate SIFT data from above data
data_types = ["train", "test"]


for data_type in data_types:
    folder_path = "./data/clothing-dataset/" + data_type
    new_folder_path = "./data/clothing-dataset-sift/" + data_type

    for folder in os.listdir(folder_path):
        new_path = os.path.join(new_folder_path, folder)
        
        if not os.path.exists(new_path):
            os.makedirs(new_path)

        for filename in os.listdir(os.path.join(folder_path, folder)):
            # Open the image
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder_path, folder, filename)
                img = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

                # Apply the SIFT algorithm
                sift = cv.SIFT_create()
                keypoints, descriptors = sift.detectAndCompute(img, None)

                # Draw the keypoints on the image
                img_with_keypoints = cv.drawKeypoints(img, keypoints, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

                # Save the image with keypoints to the new folder
                new_image_path = os.path.join(new_folder_path, folder, filename)
                cv.imwrite(new_image_path, img_with_keypoints)

In [3]:
# Feature: Grayscale
X_train = []
y_train = []
X_test = []
y_test = []

for data_type in ["train", "test"]:
    folder_path = "./data/clothing-dataset/" + data_type

    for folder in os.listdir(folder_path):
        for filename in os.listdir(os.path.join(folder_path, folder)):
            # Open the image
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder_path, folder, filename)
                img = Image.open(image_path)
                # resize the image
                img = img.resize((64, 64))
                # convert to grayscale
                img = img.convert('L')
                # flatten to 1D array
                array = np.array(img).ravel()
                
                if data_type == "test":
                    X_test.append(array)
                    y_test.append(folder)
                else:
                    X_train.append(array)
                    y_train.append(folder)

In [6]:
from skimage.io import imread, imshow
from skimage import transform
from skimage.filters import prewitt

def preprocess_image_edge_only(image):
    resized_image = transform.resize(image, (64, 64), anti_aliasing=True)
    edges_prewitt = prewitt(resized_image)
    edges_prewitt_array = edges_prewitt.reshape(1, 64 * 64)
    return edges_prewitt_array[0]

def preprocess_image(image):
    resized_image = transform.resize(image, (64, 64), anti_aliasing=True)
    edges_prewitt = prewitt(resized_image)
    edges_prewitt_array = edges_prewitt.reshape(1, 64 * 64)
    image_array = resized_image.reshape(1, 64 * 64)
    return np.concatenate((edges_prewitt_array[0], image_array[0]))

In [7]:
# Feature: Edges
X_train = []
y_train = []
X_test = []
y_test = []

data_types = ["train", "test"]

image = None
for data_type in data_types:
    folder_path = "./data/clothing-dataset/" + data_type

    for folder in os.listdir(folder_path):
        for filename in os.listdir(os.path.join(folder_path, folder)):
            # Open the image
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder_path, folder, filename)
                image = imread(image_path,as_gray=True)
                result = preprocess_image_edge_only(image)
                if data_type == "test":
                    X_test.append(result)
                    y_test.append(folder)
                else:
                    X_train.append(result)
                    y_train.append(folder)

In [9]:
# Feature: Grayscale + Edges
X_train = []
y_train = []
X_test = []
y_test = []

data_types = ["train", "test"]

image = None
count = 0
for data_type in data_types:
    folder_path = "./data/clothing-dataset/" + data_type

    for folder in os.listdir(folder_path):
        for filename in os.listdir(os.path.join(folder_path, folder)):
            # Open the image
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder_path, folder, filename)
                image = imread(image_path,as_gray=True)
                result = preprocess_image(image)
                if data_type == "test":
                    X_test.append(result)
                    y_test.append(folder)
                else:
                    X_train.append(result)
                    y_train.append(folder)

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# define the number of folds for cross validation
NUM_FOLDS = 5

# define param search space for knn and dt
knn_param_grid = {'n_neighbors': list(range(2, 100))}
dt_param_grid = {'max_depth': list(range(2, 100))}

# define models
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier(max_features=512)

# define scoring metric
f1_scorer = make_scorer(f1_score, average='micro')

In [5]:
# Experiment: Grayscale

# KNN
knn_model = RandomizedSearchCV(knn, knn_param_grid, cv=NUM_FOLDS, scoring=f1_scorer, random_state=42, n_jobs=-1)
knn_model.fit(X_train, y_train)

print("KNN:")
print("Best k: ", knn_model.best_params_)
print("Best F1 score: ", knn_model.best_score_)

knn_final = KNeighborsClassifier(n_neighbors=int(knn_model.best_params_['n_neighbors']))
knn_final.fit(X_train, y_train)
y_pred = knn_final.predict(X_test)
print("Test F1 Score: ", f1_score(y_test, y_pred, average='micro'))

# DT
dt_model = RandomizedSearchCV(dt, dt_param_grid, cv=NUM_FOLDS, scoring=f1_scorer, random_state=42, n_jobs=-1)
dt_model.fit(X_train, y_train)

print("DT:")
print("Best depth: ", dt_model.best_params_)
print("Best F1 score: ", dt_model.best_score_)

dt_final = DecisionTreeClassifier(max_depth=int(dt_model.best_params_['max_depth']), max_features=512)
dt_final.fit(X_train, y_train)
y_pred = dt_final.predict(X_test)
print("Test F1 Score: ", f1_score(y_test, y_pred, average='micro'))

KNN:
Best k:  {'n_neighbors': 2}
Best F1 score:  0.5319744968609561
Test F1 Score:  0.3439306358381503
DT:
Best depth:  {'max_depth': 12}
Best F1 score:  0.3680579271832426
Test F1 Score:  0.3265895953757225


In [8]:
# Experiment: Edges

# KNN
knn_model = RandomizedSearchCV(knn, knn_param_grid, cv=NUM_FOLDS, scoring=f1_scorer, random_state=42, n_jobs=-1)
knn_model.fit(X_train, y_train)

print("KNN:")
print("Best k: ", knn_model.best_params_)
print("Best F1 score: ", knn_model.best_score_)

knn_final = KNeighborsClassifier(n_neighbors=int(knn_model.best_params_['n_neighbors']))
knn_final.fit(X_train, y_train)
y_pred = knn_final.predict(X_test)
print("Test F1 Score: ", f1_score(y_test, y_pred, average='micro'))

# DT
dt_model = RandomizedSearchCV(dt, dt_param_grid, cv=NUM_FOLDS, scoring=f1_scorer, random_state=42, n_jobs=-1)
dt_model.fit(X_train, y_train)

print("DT:")
print("Best depth: ", dt_model.best_params_)
print("Best F1 score: ", dt_model.best_score_)

dt_final = DecisionTreeClassifier(max_depth=int(dt_model.best_params_['max_depth']), max_features=512)
dt_final.fit(X_train, y_train)
y_pred = dt_final.predict(X_test)
print("Test F1 Score: ", f1_score(y_test, y_pred, average='micro'))

KNN:
Best k:  {'n_neighbors': 2}
Best F1 score:  0.4515318591012451
Test F1 Score:  0.31358381502890176
DT:
Best depth:  {'max_depth': 12}
Best F1 score:  0.27196311304805837
Test F1 Score:  0.23699421965317918


In [10]:
# Experiment: Grayscale + Edges

# KNN
knn_model = RandomizedSearchCV(knn, knn_param_grid, cv=NUM_FOLDS, scoring=f1_scorer, random_state=42, n_jobs=-1)
knn_model.fit(X_train, y_train)

print("KNN:")
print("Best k: ", knn_model.best_params_)
print("Best F1 score: ", knn_model.best_score_)

knn_final = KNeighborsClassifier(n_neighbors=int(knn_model.best_params_['n_neighbors']))
knn_final.fit(X_train, y_train)
y_pred = knn_final.predict(X_test)
print("Test F1 Score: ", f1_score(y_test, y_pred, average='micro'))

# DT
dt_model = RandomizedSearchCV(dt, dt_param_grid, cv=NUM_FOLDS, scoring=f1_scorer, random_state=42, n_jobs=-1)
dt_model.fit(X_train, y_train)

print("DT:")
print("Best depth: ", dt_model.best_params_)
print("Best F1 score: ", dt_model.best_score_)

dt_final = DecisionTreeClassifier(max_depth=int(dt_model.best_params_['max_depth']), max_features=512)
dt_final.fit(X_train, y_train)
y_pred = dt_final.predict(X_test)
print("Test F1 Score: ", f1_score(y_test, y_pred, average='micro'))

KNN:
Best k:  {'n_neighbors': 2}
Best F1 score:  0.5511642922071853
Test F1 Score:  0.3453757225433526
DT:
Best depth:  {'max_depth': 42}
Best F1 score:  0.34668793344150783
Test F1 Score:  0.30057803468208094
